In [ ]:
!pip install opendatasets --upgrade --quiet

In [ ]:
import pandas as pd
import os
import opendatasets as od

In [ ]:
dataset = 'https://www.kaggle.com/datasets/sachinkumar413/diabetic-retinopathy-dataset'
od.download(dataset)

In [ ]:
data = '.\diabetic-retinopathy-dataset'

In [ ]:
data = os.listdir('.\diabetic-retinopathy-dataset')
Healthy = os.listdir('.\diabetic-retinopathy-dataset/Healthy')
Mild = os.listdir('.\diabetic-retinopathy-dataset/Mild DR')
Moderate = os.listdir('.\diabetic-retinopathy-dataset/Moderate DR')
Proliferate = os.listdir('.\diabetic-retinopathy-dataset/Proliferate DR')
Severe = os.listdir('.\diabetic-retinopathy-dataset/Severe DR')

In [ ]:
print("\nClasses:", (data))
print("\nNumber of classes:", len(data))
print("\nNumber of Healty images:", len(Healthy))
print("\nNumber of Mild images:", len(Mild))
print("\nNumber of Moderate images:", len(Moderate))
print("\nNumber of Proliferate images:", len(Proliferate),  "\n______________________________\n")
print("\nNumber of Severe images:", len(Severe))

In [ ]:
# Get dataset path
Path_data = '.\diabetic-retinopathy-dataset'

# Create two lists to store paths of images and their labels
img_paths = []
labels =[]

# Convert directory to list
data = os.listdir(Path_data)

# Get paths and Labels of classes and images in data
for i in data:
  class_path = os.path.join(Path_data, i)
  img_list = os.listdir(class_path)

  for img in img_list:
    img_path = os.path.join(class_path, img)

    img_paths.append(img_path)
    labels.append(i)
    
# Convert two lists of imgpaths and their labels into series
Paths = pd.Series(img_paths, name = 'Paths')
Labels = pd.Series(labels, name = 'Labels')

# Concatenate them in one Dataframe called df
df= pd.concat([Paths, Labels], axis = 1)
df.head()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
import random
import itertools

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Conv2D, MaxPooling2D, Flatten, Dense, Activation
from tensorflow .keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from PIL import Image, UnidentifiedImageError

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Train, valid  and test dataframes
train, testval = train_test_split(df, test_size = 0.2, shuffle = True, random_state = 123)
valid, test = train_test_split(testval, test_size = 0.5, shuffle = True, random_state = 123)
print("Train shape:", train.shape)
print("Valid shape:", valid.shape)
print("Test shape:", test.shape)

In [ ]:
train.Labels.value_counts()

In [ ]:
batch_size = 20
img_size = (224, 224) # Standard value (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)

#Create generators
tr_G = ImageDataGenerator(
  zca_whitening=True,
  rotation_range=30.,
  fill_mode='nearest')

V_G = ImageDataGenerator()
t_G = ImageDataGenerator()

#Generate Appropriate Data for fitting into model
Train = tr_G.flow_from_dataframe(train, x_col = 'Paths', y_col = 'Labels', target_size = img_size, class_mode = 'categorical', color_mode = 'rgb', shuffle = True, batch_size = batch_size)
Valid = V_G.flow_from_dataframe(valid, x_col = 'Paths', y_col = 'Labels', target_size = img_size, class_mode = 'categorical', color_mode = 'rgb', shuffle = True, batch_size = batch_size)
Test = t_G.flow_from_dataframe(test, x_col = 'Paths', y_col = 'Labels', target_size = img_size, class_mode = 'categorical', color_mode = 'rgb', shuffle = False, batch_size = batch_size)
     

In [ ]:
L_index = Train.class_indices
print(L_index)
Keys = list(L_index.keys())
print(Keys)
imgs, labels = next(Train)
plt.figure(figsize= (15, 15))
for i in range(8):
  plt.subplot(3, 4, i +1)
  im = imgs[i]/255
  plt.imshow(im)

  #Labelling
  index = np.argmax(labels[i])
  label = Keys[index]
  plt.title(label, color = 'purple')
  plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
#Model Preparation
n_classes = len(list(Train.class_indices.keys()))
print(n_classes)

img_shape=(img_size[0], img_size[1], 3)
model_name='EfficientNetB3'
base_model= EfficientNetB3(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max')

base_model.trainable=True
x=base_model.output
x=BatchNormalization(axis=-1, momentum=0.999, epsilon=0.001 )(x)

x = Dense(1024, kernel_regularizer = regularizers.l2(l = 0.01),activity_regularizer=regularizers.l1(0.005),
          bias_regularizer=regularizers.l1(0.005) ,activation='relu')(x)

x=Dropout(rate=.2, seed=123)(x)

x = Dense(512, kernel_regularizer = regularizers.l2(l = 0.01),activity_regularizer=regularizers.l1(0.005),
          bias_regularizer=regularizers.l1(0.005) ,activation='relu')(x)

x=Dropout(rate=.3, seed=123)(x)

x = Dense(256, kernel_regularizer = regularizers.l2(l = 0.01),activity_regularizer=regularizers.l1(0.005),
          bias_regularizer=regularizers.l1(0.005) ,activation='relu')(x)

x=Dropout(rate=.4, seed=123)(x)

output=Dense(n_classes, activation='softmax')(x)
model=Model(inputs=base_model.input, outputs=output)
lr=.0001 

In [ ]:
model = Sequential()
model.add(base_model)
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(512, activation='elu'))
model.add(Dense(256, activation='elu'))
model.add(Dense(128, activation='elu'))
model.add(Dense(5, activation='softmax'))

In [ ]:
model.compile(
  Adamax(learning_rate=0.0001),
  loss='categorical_crossentropy',
  metrics=['acc'])

In [ ]:
epochs = 50
history = model.fit(x= Train, epochs= epochs, verbose= 1, validation_data= Valid,validation_steps= None, shuffle= False)

In [ ]:
# Accuracy and loss of Train
tr_acc = history.history['acc']
tr_loss = history.history['loss']

# accuracy and loss or Valid\
v_acc = history.history['val_acc']
v_loss = history.history['val_loss']

# Highest value of v_acc by getting its index
index_acc = np.argmax(v_acc)
high_Vacc = v_acc[index_acc]

# Lowest value of v_loss by getting index
index_loss = np.argmin(v_loss)
low_Vloss = v_loss[index_loss]

# Number of epochs based on length of tr_acc values
Epochs =[]
for i in range(len(tr_acc)):
  Epochs.append (i+1)

# Define best epoch
best_acc = f'Best epoch accuracy = {str(index_acc +1)}'
best_loss = f'Best epoch loss = {str(index_loss+1)}'

print(best_acc)
print(best_loss)

In [ ]:
plt.figure(figsize = (16, 8))
plt.style.use('fivethirtyeight')

# Subplot 1 for best epoch accuracy
plt.subplot(1,2,1)
plt.plot(Epochs, tr_acc, "g", label = "Train Accuarcy")
plt.plot(Epochs, v_acc, "r", label = "Valid Accuarcy")
plt.scatter(index_acc+1, high_Vacc, s = 150, color = 'purple', label = best_acc)
plt.title("Accuracy: Train vs Valid")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Subplot2 for best epoch loss
plt.subplot(1,2,2)
plt.plot(Epochs, tr_loss, "g", label = "Train Loss")
plt.plot(Epochs, v_loss, "r", label = "Valid Loss")
plt.scatter(index_loss+1, low_Vloss, s = 150, color = 'purple', label = best_loss)
plt.title("Loss: Train vs Valid")
plt. xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
Train_scores = model.evaluate(Train, verbose = 1)
Valid_scores = model.evaluate(Valid, verbose = 1)
Test_scores = model.evaluate(Test, verbose = 1)
print('\nTrain Scores: \n    accuracy:', Train_scores[1], '\n      Loss:', Train_scores[0], '\n________________________')
print('Valid Scores: \n    accuracy:', Valid_scores[1], '\n      Loss:', Valid_scores[0], '\n________________________')
print('Test Scores: \n    accuracy:', Test_scores[1], '\n      Loss:', Test_scores[0])

In [ ]:
predictions = model.predict_generator(Test)
y_pred = np.argmax(predictions, axis = 1)

# Checking predictions
print(predictions)
print()
print(y_pred)

In [ ]:
Test_cl_ind = Test.class_indices

In [ ]:
classes = list(Test_cl_ind.keys())
confusion_matrix(Test.classes, y_pred)

In [ ]:
classification_report(Test.classes, y_pred, target_names = classes)